# Python SDK Tutorial
This notebook explains how to use the Python SDK of ModelBox and explains the major concepts and how to use the API independent of any Deep Learning Framework. Please follow the PyTorch notebook to see how the SDK can be integrated with a PyTorch trainer. 



## Install ModelBox SDK

In [7]:
pip install modelbox==0.0.3 grpcio

Note: you may need to restart the kernel to use updated packages.


## Initialize the ModelBox Client
First, we initialize the client by pointing it to the address of the ModelBox Server

In [1]:
from modelbox.modelbox import ModelBoxClient, MLFramework, Artifact, ArtifactMime, MetricValue

client = ModelBoxClient(addr="172.21.0.2:8085")

## Create an Experiment 
Once we have a client, we can start using it to create a new Experiment to train a model or track an existing pre-trained model. Let us first see how to create an experiment. We are going to create an experiment to train a Wav2Vec Model with Pytorch and store it in a namespace called *langtech*. If you are using an experiment management service like Weights and Biases or Nepute, you could associate the ID from that service with modelbox to create a lineage.

In [2]:
resp = client.create_experiment("wav2vec", "owner@pytorch.org", "langtech", "extern123", MLFramework.PYTORCH)
experiment_id = resp.experiment_id
experiment_id

'e71a25a8d3d481463c281f083f5e2671ea2896bc'

The above code is going to create a new experiment and give us an ID. You can list the experiments of a namespace by -

In [3]:
resp = client.list_experiments(namespace="langtech")
resp.experiments

[Experiment(id='e71a25a8d3d481463c281f083f5e2671ea2896bc', name='wav2vec', owner='owner@pytorch.org', namespace='langtech', external_id='', created_at=seconds: 1663448562
 , updated_at=seconds: 1663448562
 )]

#### Adding metadata
Metadata can be added to any of the objects in ModelBox after they have been created. For example, once an experiment is created, metadata can be added and listed at any stage -

In [4]:
meta_resp = client.update_metadata(experiment_id, "foo/bar", 12)
resp = client.list_metadata(experiment_id)
resp

ListMetadataResponse(metadata={'foo/bar': number_value: 12
})

## Working with Checkpoints
Once we have an experiment we can create model checkpoints from the trainers. Let's assume the file stored in assets/mnist_checkpoint1.pt is a checkpoint created by the trainer. We will now associate this checkpoint with ModelBox.

We could either track the path of the checkpoint or upload the blob and let ModelBox store it in the configured blob store. The benefit of letting ModelBox store the checkpoint is that the trainer doesn't need to have access to the blob store directly. However, in some cases, it's more optimal to have the trainer store the blob directly when the path to IO to the blob store from the trainer is much faster.

In [5]:
import os
checkpoint_path = os.path.abspath('mnist_cnn_checkpoint1.pt')
metrics = {'val_accu': 98.5, 'train_accu': 99.2}
resp = client.create_checkpoint(experiment=experiment_id, epoch=1, path=checkpoint_path, metrics=metrics)
checkpoint_id = resp.checkpoint_id
checkpoint_id

'97f7fccf8d2d67aa78d7d58a86b5d938ddff9f38'

This returns us the checkpoint ID and tracks the path of the checkpoint created by the trainer.

Now let's say that we also want ModelBox to store the checkpoint, we will simply set the flag `upload` in the above api

In [12]:
#resp = client.create_checkpoint(experiment_id, 2, checkpoint_path, metrics, upload=True)

Once checkpoints are created they can be listed by passing the experiment name

In [6]:
client.list_checkpoints(experiment_id=experiment_id)

ListCheckpointsResponse(checkpoints=[Checkpoint(id='97f7fccf8d2d67aa78d7d58a86b5d938ddff9f38', experiment_id='e71a25a8d3d481463c281f083f5e2671ea2896bc', epoch=1)])

## Working with Models and ModelVersions

Model objects describe tasks performed, metadata, which datasets are used to train, how to use the models during inference, etc. ModelVersions are trained instances of a model. So for example over time an English ASR(speech to text) model can have multiple model versions as they are trained with different datasets and such. 

We don't prescribe the granularity of Models and ModelVersions. If it's easier to create different Models every time a new model is trained with different hyperparameters and a single ModelVersion pointing to the model artifacts and all the metrics that is fine.

In [6]:
resp = client.create_model(name='asr_en', owner='owner@owner.org', namespace='langtech', task='asr', description='ASR for english', metadata={'x': 'y'})
model_id = resp.id
model_id

'93b7cf0bec10ff6d500a9273bc34ab7eff02eeca'

In the same way a ModelVersion can be created by the client, and track the associated artifacts and metadata.


In [8]:
tags =["test"]
resp = client.create_model_version(model_id=model_id, name="asr_en_july", version="1", description='ASR for english', metadata={'x': 'y'}, unique_tags=tags, files=[], framework=MLFramework.PYTORCH.to_proto())
model_version_id = resp.id
model_version_id

'0ae662556996e18f364a41b82cd1a4e043062687'

Once a modelversion is created we can upload the model and associate with the model version object.

In [11]:
model_path = './artifacts/mnist_cnn.pt'
resp = client.upload_artifact(parent=model_id, path=model_path, artifact_type=ArtifactMime.Checkpoint)
resp

UploadArtifactResponse(id='226cd4e3abacc813881706fd03f0a1e19afb0cc6')

The model file can now be served by the file server built into Model Box to inference servers. Inference services can either use the language specific SDKs in Python, Rust or Go or call the GRPC `DownloadFile` API directly which streams the files.

> **_NOTE: Checkpoints Transforms to ModelVersions_** 
Usually in production engineers look at checkpoints/models created during training and select a version which has the best metrics. Once we have the worker infrastructure in place, we will create APIs which to do automatic convertion of checkpoints to ModelVersions.

## Tracking Artifacts and Working with Files
Modelbox can track artifacts used in training and also users can upload Files and associate them with experiments, models and model versions. For example, a user can track the dataset files used for training stored in S3 or even upload them to ModelBox. A trained model can be uploaded and then later streamed to applications for inferencing.


In [12]:
import pathlib
file_path = str(
            pathlib.Path(".").parent.resolve().joinpath("artifacts/test_artifact.txt")
        )
artifacts = [Artifact(parent='93b7cf0bec10ff6d500a9273bc34ab7eff02eeca', mime_type=ArtifactMime.Text, path=file_path)]
resp = client.track_artifacts(artifacts=artifacts)
resp

TrackArtifactsResponse(num_artifacts_tracked=1)

Modelbox is now tracking the artifact and has information about the checksum, local path of the file, etc.

In [13]:
result = client.list_models(namespace='langtech')
artifacts = result.models[0].artifacts
artifacts

[Artifact(parent='93b7cf0bec10ff6d500a9273bc34ab7eff02eeca', path='/tmp/modelboxblobs/226cd4e3abacc813881706fd03f0a1e19afb0cc6', mime_type=<ArtifactMime.Checkpoint: 2>, checksum='31c437e7d87fa749a7e049f0ccc46dd0', id='226cd4e3abacc813881706fd03f0a1e19afb0cc6'),
 Artifact(parent='93b7cf0bec10ff6d500a9273bc34ab7eff02eeca', path='/home/jovyan/work/tutorials/artifacts/test_artifact.txt', mime_type=<ArtifactMime.Text: 3>, checksum='0019d23bef56a136a1891211d7007f6f', id='c0591638c4a3111bfbf84324761cad0de56592ec')]

## Metrics 
ModelBox supports adding Metrics to experiments. Metrics can be logged to a key with values being a float, string or bytes. Metric values are associated with a step unit, and wallclock time when the metric was emitted by the application.

In [14]:
import time
client.log_metrics(parent_id=experiment_id, key="val_accu", value=MetricValue(step=1, wallclock_time=int(time.time()), value=0.73))
client.log_metrics(parent_id=experiment_id, key="val_accu", value=MetricValue(step=2, wallclock_time=int(time.time()), value=0.78))

metrics = client.get_metrics(experiment_id)
metrics

{'val_accu': Metrics(key='val_accu', values=[[MetricValue(step=1, wallclock_time=1663395823, value=0.7300000190734863), MetricValue(step=2, wallclock_time=1663395823, value=0.7799999713897705)]])}

Metrics can be added to any of the modelbox objects including Model, ModelVersion.